In [2]:
import pandas as pd
import matplotlib.pyplot as plt

# Extract Data

In [3]:
file_path = 'data/all_minus_med.csv'
df = pd.read_csv(file_path)

pd.set_option('display.max_columns', None)
df.head()

,Course ID,Class Nbr,Subject,Catalog,Descr,PI Name,Course Long Descr,Section,Component,Mode,Max Units,Mtg Start,Mtg End,Pat,Start Date,End Date,Descr 1,Attribute Formal Desc,Term Descr,Career,Location
0,19,8275,AMES,165S,THE WORLD OF JAPANESE POP CULT,"Maude,Daryl J",An examination of modern Japanese culture thro...,01,SEM,In Person,1.0,11:45:00.000000AM,1:00:00.000000PM,TTH,8/28/23,12/8/23,Languages 207,"(ALP) Arts, Literature & Performance",2023 Fall Term,UGRD,DURHAM
1,19,8275,AMES,165S,THE WORLD OF JAPANESE POP CULT,"Maude,Daryl J",An examination of modern Japanese culture thro...,01,SEM,In Person,1.0,11:45:00.000000AM,1:00:00.000000PM,TTH,8/28/23,12/8/23,Languages 207,(CCI) Cross Cultural Inquiry,2023 Fall Term,UGRD,DURHAM
2,19,8275,AMES,165S,THE WORLD OF JAPANESE POP CULT,"Maude,Daryl J",An examination of modern Japanese culture thro...,01,SEM,In Person,1.0,11:45:00.000000AM,1:00:00.000000PM,TTH,8/28/23,12/8/23,Languages 207,(CZ) Civilizations,2023 Fall Term,UGRD,DURHAM
3,19,8275,AMES,165S,THE WORLD OF JAPANESE POP CULT,"Maude,Daryl J",An examination of modern Japanese culture thro...,01,SEM,In Person,1.0,11:45:00.000000AM,1:00:00.000000PM,TTH,8/28/23,12/8/23,Languages 207,Crosslisted in another department,2023 Fall Term,UGRD,DURHAM
4,19,8275,AMES,165S,THE WORLD OF JAPANESE POP CULT,"Maude,Daryl J",An examination of modern Japanese culture thro...,01,SEM,In Person,1.0,11:45:00.000000AM,1:00:00.000000PM,TTH,8/28/23,12/8/23,Languages 207,Seminar,2023 Fall Term,UGRD,DURHAM


# Transform Data

In [4]:
# Extract all engineering-related courses
eng_subjects = ['BME', 'COMPSCI', 'ECE', 'ME', 'PHYSICS', 'EGR', 'CHEM', 'NANOSCI', 'EGRMGMT', 'MENG', 'FINTECH', 'CYBERSEC', 'AIPI', 'GAMEDSGN']

# get all courses that are in eng_subjects
eng_courses = df[df['Subject'].isin(eng_subjects)]

print(eng_courses.shape)
eng_courses.head(10)

(7253, 21)


,Course ID,Class Nbr,Subject,Catalog,Descr,PI Name,Course Long Descr,Section,Component,Mode,Max Units,Mtg Start,Mtg End,Pat,Start Date,End Date,Descr 1,Attribute Formal Desc,Term Descr,Career,Location
1605,1649,5098,BME,271D,SIGNALS AND SYSTEMS,"Rizk,Michael","Convolution, deconvolution, Fourier series, Fo...",001,LEC,In Person,1.0,8:30:00.000000AM,9:45:00.000000AM,MW,1/11/23,4/26/23,Hudson Hall 125,Graded only. S/U not allowed.,2023 Spring Term,UGRD,DURHAM
1606,1649,5098,BME,271D,SIGNALS AND SYSTEMS,"Rizk,Michael","Convolution, deconvolution, Fourier series, Fo...",001,LEC,In Person,1.0,8:30:00.000000AM,9:45:00.000000AM,MW,1/11/23,4/26/23,Hudson Hall 125,Undergraduate Bulletin,2023 Spring Term,UGRD,DURHAM
1607,1649,5099,BME,271D,SIGNALS AND SYSTEMS,"Rizk,Michael","Convolution, deconvolution, Fourier series, Fo...",01D,DIS,In Person,1.0,1:45:00.000000PM,3:00:00.000000PM,M,1/11/23,4/26/23,Hudson Hall 212,Graded only. S/U not allowed.,2023 Spring Term,UGRD,DURHAM
1608,1649,5099,BME,271D,SIGNALS AND SYSTEMS,"Rizk,Michael","Convolution, deconvolution, Fourier series, Fo...",01D,DIS,In Person,1.0,1:45:00.000000PM,3:00:00.000000PM,M,1/11/23,4/26/23,Hudson Hall 212,Undergraduate Bulletin,2023 Spring Term,UGRD,DURHAM
1609,1649,7414,BME,271D,SIGNALS AND SYSTEMS,"Bucholz,Elizabeth K","Convolution, deconvolution, Fourier series, Fo...",001,LEC,In Person,1.0,8:30:00.000000AM,9:45:00.000000AM,MW,8/28/23,12/8/23,Wilkinson 126,Graded only. S/U not allowed.,2023 Fall Term,UGRD,DURHAM
1610,1649,7414,BME,271D,SIGNALS AND SYSTEMS,"Bucholz,Elizabeth K","Convolution, deconvolution, Fourier series, Fo...",001,LEC,In Person,1.0,8:30:00.000000AM,9:45:00.000000AM,MW,8/28/23,12/8/23,Wilkinson 126,Undergraduate Bulletin,2023 Fall Term,UGRD,DURHAM
1611,1649,7415,BME,271D,SIGNALS AND SYSTEMS,"Bucholz,Elizabeth K","Convolution, deconvolution, Fourier series, Fo...",01D,DIS,In Person,1.0,10:05:00.000000AM,11:20:00.000000AM,F,8/28/23,12/8/23,Wilkinson 136,Graded only. S/U not allowed.,2023 Fall Term,UGRD,DURHAM
1612,1649,7415,BME,271D,SIGNALS AND SYSTEMS,"Bucholz,Elizabeth K","Convolution, deconvolution, Fourier series, Fo...",01D,DIS,In Person,1.0,10:05:00.000000AM,11:20:00.000000AM,F,8/28/23,12/8/23,Wilkinson 136,Undergraduate Bulletin,2023 Fall Term,UGRD,DURHAM
1613,1649,7416,BME,271D,SIGNALS AND SYSTEMS,"Bucholz,Elizabeth K","Convolution, deconvolution, Fourier series, Fo...",02D,DIS,In Person,1.0,1:25:00.000000PM,2:40:00.000000PM,F,8/28/23,12/8/23,Wilkinson 132,Graded only. S/U not allowed.,2023 Fall Term,UGRD,DURHAM
1614,1649,7416,BME,271D,SIGNALS AND SYSTEMS,"Bucholz,Elizabeth K","Convolution, deconvolution, Fourier series, Fo...",02D,DIS,In Person,1.0,1:25:00.000000PM,2:40:00.000000PM,F,8/28/23,12/8/23,Wilkinson 132,Undergraduate Bulletin,2023 Fall Term,UGRD,DURHAM


# Load

In [5]:
# Load into a database

#### Store descriptions in vector database

In [9]:
# print a few of the 'Course Long Descr' randomly
import random
for i in range(10):
    print(random.choice(eng_courses['Course Long Descr'].values))
    print()

Laboratory component of Physics 152L.

Individual research projects in engineering under the guidance and supervision of a faculty member, the central goal of which is a substantive paper or written report containing significant analysis and interpretation of a previously approved topic. Consent of instructor required.

Lab component for Chemistry 110DL.

Review basic concepts of financial accounting and finance, with an emphasis on accounting needed for effective financial analysis. Focus on issues of finance in high tech industries. Emphases will include project financing, notions of options as applied to internal financial analysis, allocation of costs and revenues for new high tech projects, valuing projects and valuing firms when intellectual assets are a significant portion of total level value; corporate control in high tech firms. Finance issues in mergers, acquisitions, and alliances.  Prerequisite: enrollment in the Master of Engineering Management Program.

Computer structur